# 極大ナップサック問題

## 問題

正整数の配列 $a = [ a_1, \dots, a_n]$ と整数 $\tau$ が与えられる. 
このとき $a$ の部分配列でその**総積**が $\tau$ 以下になるもので極大なものを全て列挙せよ. 

## 定式化

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする. 
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない. 
また, 決定変数 $y = [y_1, \dots, y_n]$ を用意し $y_i = (a_i - 1) x_i + 1$ を課す. 
こうすると $y_i$ は $x_i$ が $1$ のとき $a_i$ を値に取り, $0$ のとき $1$ を値にとる. 
同様に $z_i = (a_i - 1) (1 - x_i) + 1$ とおく. 
こちらは $y_i$ とは逆に採用されてない $a_i$ に対してだけ値 $a_i$ を取り, 採用されているものについては $1$ を取る. 

Google OR-Tools は整数変数の積をそのまま扱えるため, 線形ではない定式化を行う. 

- $\prod_{i=1}^n y_i \leq \tau$: 採用した $a_i$ の積が $\tau$ を超えない. 
- $z_j \prod_{i=1}^n y_i > \tau (1 - x_j) \quad (\forall j = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう. 

また, Google OR-Tools の CP-SAT ソルバーは目的関数を設定しない場合, 
実行可能解を全て求めるよう指示できるのでそれを使って全ての極大元を求める. 

## 実装

In [1]:
from ortools.sat.python import cp_model
import math
import time

In [2]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, a, solution: list[cp_model.IntVar]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__solution = solution
        self.__a = a
        self.__solution_count = 0
        self.__all_solutions = []
        self.__start_time = time.time()

    @property
    def solution_count(self) -> int:
        return self.__solution_count

    @property
    def all_solutions(self):
        return self.__all_solutions

    def on_solution_callback(self):
        current_time = time.time()
        self.__solution_count += 1
        print(
            f"Solution {self.__solution_count}, "
            f"time = {current_time - self.__start_time} s"
        )

        print("  [", end=' ')
        for x in self.__solution:
            print(f"{self.value(x)}", end=' ')
        print("]")
        print("  [", end=' ')
        solution = []
        for i, x in enumerate(self.__solution):
            if self.value(x) == 1:
                solution.append(self.__a[i])
                print(f"{self.__a[i]}", end=' ')
        print("]")
        self.__all_solutions.append(solution)

In [3]:
class Model:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau

        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()

        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        y = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]
        z = [self.model.new_int_var(1, self.a[i], "") for i in range(len(self.a))]

        for i in range(len(self.a)):
            self.model.add(y[i] == (self.a[i] - 1) * self.x[i] + 1)
            self.model.add(z[i] == (self.a[i] - 1) * (1 - self.x[i]) + 1)

        y_prod = self.model.new_int_var(1, tau, "")
        self.model.add_multiplication_equality(y_prod, y)
        for j in range(len(self.a)):
            y_prod_zj = self.model.new_int_var(1, tau * self.a[j], "")
            self.model.add_multiplication_equality(y_prod_zj, [z[j], y_prod])
            self.model.add(y_prod_zj >= self.tau * (1 - self.x[j]) + 1)

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [4]:
# a = list(range(1, 5+1))
# tau = 10

# a = list(range(1, 10+1))
# tau = 50

a = list(range(1, 20+1))
tau = 100

# a = [2] * 20 # 終わらない
# tau = 2 ** 10

print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [5]:
model = Model(a, tau)
model.solve()

Solution 1, time = 0.007972955703735352 s
  [ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 ]
  [ 1 2 20 ]
Solution 2, time = 0.017207860946655273 s
  [ 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 ]
  [ 1 2 19 ]
Solution 3, time = 0.024940967559814453 s
  [ 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 6 7 ]
Solution 4, time = 0.030471086502075195 s
  [ 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 6 ]
Solution 5, time = 0.03348803520202637 s
  [ 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 5 ]
Solution 6, time = 0.03737998008728027 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 ]
  [ 1 4 13 ]
Solution 7, time = 0.041252851486206055 s
  [ 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 2 3 8 ]
Solution 8, time = 0.04309701919555664 s
  [ 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 5 14 ]
Solution 9, time = 0.04436492919921875 s
  [ 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [ 1 4 14 ]
Solution 10, time = 0.04537200927734375 s
  [ 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 ]
  [

In [6]:
model.solution_printer.all_solutions

[[1, 2, 20],
 [1, 2, 19],
 [1, 2, 6, 7],
 [1, 2, 3, 6],
 [1, 2, 3, 5],
 [1, 4, 13],
 [1, 2, 3, 8],
 [1, 5, 14],
 [1, 4, 14],
 [1, 6, 14],
 [1, 2, 3, 4],
 [1, 6, 9],
 [1, 2, 5, 9],
 [1, 2, 4, 9],
 [1, 2, 3, 9],
 [1, 5, 15],
 [1, 5, 12],
 [1, 5, 13],
 [1, 5, 20],
 [1, 2, 4, 5],
 [1, 3, 4, 6],
 [1, 3, 20],
 [1, 3, 4, 8],
 [1, 6, 12],
 [1, 3, 17],
 [1, 2, 4, 12],
 [1, 6, 13],
 [1, 2, 4, 6],
 [1, 2, 4, 11],
 [1, 2, 17],
 [1, 2, 3, 15],
 [1, 2, 4, 10],
 [1, 2, 3, 12],
 [1, 2, 3, 10],
 [1, 2, 3, 14],
 [1, 2, 18],
 [1, 2, 4, 8],
 [1, 2, 5, 6],
 [1, 2, 6, 8],
 [1, 2, 5, 8],
 [1, 2, 5, 10],
 [1, 4, 15],
 [1, 4, 20],
 [1, 9, 10],
 [1, 6, 10],
 [1, 8, 10],
 [1, 4, 18],
 [1, 8, 12],
 [1, 2, 3, 13],
 [1, 2, 3, 16],
 [1, 2, 3, 11],
 [1, 3, 19],
 [1, 4, 19],
 [1, 4, 17],
 [1, 2, 4, 7],
 [1, 7, 8],
 [1, 7, 10],
 [1, 7, 14],
 [1, 7, 9],
 [1, 8, 9],
 [1, 4, 16],
 [1, 6, 16],
 [1, 6, 15],
 [1, 5, 16],
 [1, 5, 17],
 [1, 5, 19],
 [1, 3, 4, 5],
 [1, 3, 5, 6],
 [1, 2, 3, 7],
 [1, 7, 13],
 [1, 3, 4, 7],
 [1, 7

In [7]:
print("Statistics")
print(f"  conflicts      : {model.solver.num_conflicts}")
print(f"  branches       : {model.solver.num_branches}")
print(f"  wall time      : {model.solver.wall_time} s")
print(f"  solutions found: {model.solution_printer.solution_count}")

Statistics
  conflicts      : 130
  branches       : 12601
  wall time      : 0.44348800000000005 s
  solutions found: 80


## 結果

掛け算を使った定式化は厳密ではあるものの数値が大きくなりすぎてしまう問題がある. 
おとなしく $\log$ を使って線形計画問題にしよう. 

# 整数線形計画問題としての定式化

## 問題

上と同じだが一応載せておく. 

> 正整数の配列 $a = [ a_1, \dots, a_n]$ と整数 $\tau$ が与えられる.
> このとき $a$ の部分配列でその総積が $\tau$ 以下になるもので極大なものを全て列挙せよ. 

## 定式化

$x = [x_1, \dots, x_n]$ を 0-1 決定変数とする. 
$x_i$ が $1$ のとき $a_i$ を採用し, $0$ のとき採用しない. 

$$
w := \log(a_1) x_1 + \dots + \log(a_n) x_n
$$

とすると制約条件は以下のように書ける. 

- $w \leq \log(\tau)$: 採用した $a_i$ の積が $\tau$ を超えない
- $w + \log(a_i) (1 - x_i) > \log(\tau) (1 - x_i) \quad (\forall i = 1, \dots, n)$: 採用しなかった $a_j$ を掛けたら $\tau$ を超えてしまう. 

CP-SAT ソルバーは整数しか扱えないので $\log(a_i)$ や $\log(\tau)$ は適当にスケールして小数点以下を切り捨てることにする.
これによって誤差が出るかもしれない.

## 実装

In [8]:
class ModelLinear:
    def __init__(self, a, tau):
        self.a = a
        self.tau = tau
        base = 1000000
        self.log_a = [math.floor(math.log(ai) * base) for ai in a]
        self.log_tau = math.floor(math.log(self.tau) * base)
        print(f"log_a = {self.log_a}")
        print(f"log_tau = {self.log_tau}")
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.x = [self.model.new_bool_var(f"a{i} is used") for i in range(len(self.a))]
        w = sum(a * x for a, x in zip(self.log_a, self.x))

        self.model.add(w <= self.log_tau)
        for a, x in zip(self.log_a, self.x):
            self.model.add(w + a * (1 - x) >= (self.log_tau + 1) * (1 - x))

    def solve(self):
        self.solution_printer = SolutionPrinter(self.a, self.x)
        self.solver.parameters.enumerate_all_solutions = True
        #self.solver.parameters.log_search_progress = True
        self.solver.solve(self.model, self.solution_printer)

In [9]:
print(f"a = {a}")
print(f"tau = {tau}")

a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
tau = 100


In [10]:
model = ModelLinear(a, tau)
model.solve()

log_a = [0, 693147, 1098612, 1386294, 1609437, 1791759, 1945910, 2079441, 2197224, 2302585, 2397895, 2484906, 2564949, 2639057, 2708050, 2772588, 2833213, 2890371, 2944438, 2995732]
log_tau = 4605170
Solution 1, time = 0.0011470317840576172 s
  [ 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 5 ]
Solution 2, time = 0.0012500286102294922 s
  [ 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 6 ]
Solution 3, time = 0.001341104507446289 s
  [ 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 5 6 ]
Solution 4, time = 0.0014309883117675781 s
  [ 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 7 ]
Solution 5, time = 0.0015430450439453125 s
  [ 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 3 4 8 ]
Solution 6, time = 0.0016338825225830078 s
  [ 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 7 8 ]
Solution 7, time = 0.0017189979553222656 s
  [ 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 ]
  [ 1 6 9 ]
Solution 8, time = 0.0017938613891601562 s
  [ 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0

In [11]:
model.solution_printer.all_solutions

[[1, 3, 4, 5],
 [1, 3, 4, 6],
 [1, 3, 5, 6],
 [1, 3, 4, 7],
 [1, 3, 4, 8],
 [1, 7, 8],
 [1, 6, 9],
 [1, 7, 9],
 [1, 8, 9],
 [1, 6, 10],
 [1, 7, 10],
 [1, 8, 10],
 [1, 9, 10],
 [1, 5, 11],
 [1, 6, 11],
 [1, 7, 11],
 [1, 8, 11],
 [1, 9, 11],
 [1, 5, 12],
 [1, 6, 12],
 [1, 7, 12],
 [1, 8, 12],
 [1, 4, 13],
 [1, 5, 13],
 [1, 6, 13],
 [1, 7, 13],
 [1, 4, 14],
 [1, 5, 14],
 [1, 6, 14],
 [1, 7, 14],
 [1, 4, 15],
 [1, 5, 15],
 [1, 6, 15],
 [1, 4, 16],
 [1, 5, 16],
 [1, 6, 16],
 [1, 3, 17],
 [1, 4, 17],
 [1, 5, 17],
 [1, 3, 18],
 [1, 4, 18],
 [1, 5, 18],
 [1, 3, 19],
 [1, 4, 19],
 [1, 5, 19],
 [1, 3, 20],
 [1, 4, 20],
 [1, 5, 20],
 [1, 2, 4, 5],
 [1, 2, 4, 6],
 [1, 2, 5, 6],
 [1, 2, 4, 7],
 [1, 2, 5, 7],
 [1, 2, 6, 7],
 [1, 2, 4, 8],
 [1, 2, 5, 8],
 [1, 2, 6, 8],
 [1, 2, 4, 9],
 [1, 2, 5, 9],
 [1, 2, 4, 10],
 [1, 2, 5, 10],
 [1, 2, 4, 11],
 [1, 2, 4, 12],
 [1, 2, 17],
 [1, 2, 18],
 [1, 2, 19],
 [1, 2, 20],
 [1, 2, 3, 5],
 [1, 2, 3, 6],
 [1, 2, 3, 7],
 [1, 2, 3, 8],
 [1, 2, 3, 9],
 [1, 2, 3, 10]

In [12]:
print("Statistics")
print(f" conflicts : {model.solver.num_conflicts}")
print(f" branches : {model.solver.num_branches}")
print(f" wall time : {model.solver.wall_time} s")
print(f" solutions found: {model.solution_printer.solution_count}")

Statistics
 conflicts : 14
 branches : 1186
 wall time : 0.008007 s
 solutions found: 80
